<a href="https://colab.research.google.com/github/Rahul081203/ML_projects/blob/master/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer # to convert the textual data to numeric data
from sklearn.metrics.pairwise import cosine_similarity # to match features

#Data Loading and analysis

In [ ]:
data=pd.read_csv('/content/movies.csv')
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
data.describe() # movie dataframe stats

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [ ]:
data.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

#Extracting most useful Features (Data Filtering)

In [ ]:
features=['index','genres','keywords','original_language','title','cast','director']


In [ ]:
for i in data:
  if i not in features:
    data.drop(i,axis=1,inplace=True)

In [ ]:
data.dtypes

index                 int64
genres               object
keywords             object
original_language    object
title                object
cast                 object
director             object
dtype: object

In [ ]:
data.isnull().sum()

index                  0
genres                28
keywords             412
original_language      0
title                  0
cast                  43
director              30
dtype: int64

#Eliminating Null values

In [ ]:
for feature in features:
  data[feature]=data[feature].fillna('')

In [ ]:
data.isnull().sum()

index                0
genres               0
keywords             0
original_language    0
title                0
cast                 0
director             0
dtype: int64

In [ ]:
combined=""
for i in features:
  if i!='index':  
    combined+=data[i]
    if features.index(i)==(len(features)-1):
      combined+=" "


#Vectorizing the data

In [ ]:
vectorizer=TfidfVectorizer() # convert the textual data to numeric data
feature_vectors=vectorizer.fit_transform(combined)
feature_vectors # print the numeric data

<4803x28760 sparse matrix of type '<class 'numpy.float64'>'
	with 92972 stored elements in Compressed Sparse Row format>

<1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

#Cosine Similarity
<hr><br>
What cosine similarity does is that it compares the elements one by one with each and every other element in the feature vectors and the similarity index is generated as such.
Each matrix in a similarity matrix defines the similarity of a particular movie with the others.

In [ ]:
similarity=cosine_similarity(feature_vectors) # getting the similarity vectors
print(similarity)
print(similarity.shape)

[[1.         0.02034953 0.01741949 ... 0.         0.         0.        ]
 [0.02034953 1.         0.00823012 ... 0.04156584 0.         0.        ]
 [0.01741949 0.00823012 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.04156584 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(4803, 4803)


#Defining a function to match the movie

In [ ]:
# defining a function for matching movies' similarity score
def match_movie(movie,data,similarity):
  titles=data['title'].tolist()
  close_match=difflib.get_close_matches(movie,titles) # to match the movie name even if wrongly typed
  print(f"Closest match is: {close_match[0]}")
  index_of_movie=data[data.title==close_match[0]]['index'].values[0] #Finding the index of the movie in the dataset
  enumerated_matches=list(enumerate(similarity[index_of_movie]))
  sorted_matches=sorted(enumerated_matches,key=lambda x:x[1],reverse=True) # Sorting the similarity values in descending order
  print("Some other recommended movies for you are: ")
  recommended=[]
  for i in range(1,22):
    recommended+=[data[data.index==sorted_matches[i][0]]['title'].values[0]]
  for i in recommended:
    print(i)

#Frontend of the program

In [ ]:
movie=input("Enter the name of a movie you'd like to see: ")
match_movie(movie,data,similarity)

Enter the name of a movie you'd like to see: X-Men
Closest match is: X-Men
Some other recommended movies for you are: 
X2
X-Men: The Last Stand
X-Men: Apocalypse
X-Men: Days of Future Past
Iron Man 2
Avengers: Age of Ultron
Deadpool
The Avengers
Ant-Man
The Amazing Spider-Man 2
Thor: The Dark World
The Incredible Hulk
Man of Steel
Swordfish
The Image Revolution
Batman v Superman: Dawn of Justice
Superman II
The Wolverine
Sin City
Spawn
Kick-Ass 2
